In [1]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import time
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.palettes import BuPu, Greens, Set3
from scipy.fftpack import fft, rfft
import math

In [2]:
output_notebook()
opts = dict(plot_width=800, plot_height=400, min_border=0)

Loading BokehJS ...

In [3]:
def countDown():
    print 'Recording Audio for 2 seconds'
    for i in range(3, 0, -1):
        time.sleep(1)
        print i
    print 'Start'

In [4]:
fs=1024
duration = 2  #In seconds

In [5]:
#Record single - returns audio clip
def recordSingle():
    countDown()

    #Record
    myRecording1 = sd.rec(duration * fs, samplerate=fs, channels=1, dtype='int32')
    sd.wait()

    #Play
    print "Audio recording complete , Play Audio"
    sd.play(myRecording1, fs)
    sd.wait()
    
    #Each sample is of type list. Convert from list to int
    bClip = []
    for i in range(len(myRecording1)):
        bClip.append( myRecording1[i][0] )
    
    return bClip

In [6]:
#Record multiple clips at once - returns list 
def recordMultiple(numberOfSamples):
    myRecordings = []
    
    for i in range(numberOfSamples):
        myRecordings.append( recordSingle() )
    
    return myRecordings   

In [ ]:
#Record and plot single
p1 = figure(**opts)

aClip = recordSingle()

p1.line(range(len(aClip)), aClip, color="#2222aa", line_width=3)

show(p1)

In [28]:
#Record and plot multiple
p2 = figure(**opts)

aClips = recordMultiple(5)

for i in range(len(aClips)):
    p2.line(range(len(aClips[i])), aClips[i], color=Set3[9][i], line_width=1)

show(p2)

Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio
Recording Audio for 2 seconds
3
2
1
Start
Audio recording complete , Play Audio


In [29]:
#FFT

p3 = figure(**opts)

# Number of sample points
N = len(aClips[0])
# sample spacing
T = 1.0 / 1024.0

#Create axis
x = np.linspace(0.0, N*T, N)
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
xf = range(2048)

ffts = []
for i in range(len(aClips)):
    yf = rfft(aClips[i])
    
    #Scale between 0 and 1
    yfNorm = (yf-min(yf))/(max(yf)-min(yf))
    
    #Save for later
    ffts.append(yfNorm)

    p3.line(xf, ffts[i], color=BuPu[9][i], line_width=1, alpha=0.4)

show(p3)  

In [30]:
#Save data - Appends to file

#FileName of output file
fileName = 'meow_malia_1024Hz.data'

#Data to save - usually ffts or aClips
dataToSave = ffts

#Woof is always 0 1
#Meow is always 1 0
outputList = np.array([0.0, 1.0])

for i in range(len(dataToSave)):
    
    #Add output fields to match format
    dataToSave[i] = np.append(dataToSave[i], outputList)
    
    f = open(fileName,'a')
    np.savetxt(f, [dataToSave[i]], delimiter=' ', fmt='%4.6f')
    f.close()
    

In [ ]:
print ('Nips: %i') %(len(ffts[0]) - len(outputList))